In [1]:
import fer

In [2]:
import data

2024-09-25 08:37:32.653857: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-25 08:37:32.685332: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import node

In [4]:
import numpy

In [5]:
import keras

In [6]:
import constant

In [20]:
def test():
    import pandas
    pandas.read_csv(constant.train_data_csv_file_path)
    # pandas.read_csv(constant.train_data_csv_file_path)
test()

In [27]:
import numpy
def keras_inputs_and_labels_from_data_frame(a_data_frame, input_column_name='image',label_column_name='emotion'):
    get_all_inputs_as_array_from_matrix_series = lambda matrix_series: numpy.stack([ matrix for matrix in matrix_series ])
    train_inputs_as_array = get_all_inputs_as_array_from_matrix_series(train_data_frame[input_column_name])
    train_labels_as_one_hot_encoded = keras.utils.to_categorical(train_data_frame[label_column_name].to_numpy())
    return train_inputs_as_array, train_labels_as_one_hot_encoded

In [28]:
# train_inputs_as_array_and_train_labels_as_one_hot_encoded = keras.utils.image_dataset_from_directory(
#     directory='../data_png/train/',
#     labels='inferred',
#     label_mode='categorical',
#     color_mode='grayscale',
#     batch_size=32,
#     image_size=(48,48),
# )
# # dataset genreator of (32*input, 32*label)
# # train_labels_as_one_hot_encoded = None

In [29]:
train_data_frame = fer.get_data_from_path(
    data_path=constant.train_data_csv_file_path,
    # data_path='../data_csv/train/data.csv',
    input_image_format='numerals',
    output_image_format='matrix',
    sampling_quantity=None,
)
train_inputs_as_array, train_labels_as_one_hot_encoded = keras_inputs_and_labels_from_data_frame( train_data_frame )

In [30]:
# validate_inputs_as_array_and_validate_labels_as_one_hot_encoded = keras.utils.image_dataset_from_directory(
#     directory='../data_png/validate/',
#     labels='inferred',
#     label_mode='categorical',
#     color_mode='grayscale',
#     batch_size=32,
#     image_size=(48,48),
# )
# # dataset genreator of (32*input, 32*label)
# validate_labels_as_one_hot_encoded = None

In [31]:
validate_data_frame = fer.get_data_from_path(
    data_path=constant.validate_data_csv_file_path,
    input_image_format='numerals',
    output_image_format='matrix',
    sampling_quantity=None,
)
vallidate_inputs_as_array, vallidate_labels_as_one_hot_encoded = keras_inputs_and_labels_from_data_frame( vallidate_data_frame )

In [32]:
def path_data_graph_from_data_nodes(datas):
    first_data_node = node.DataNode(data=datas[0] ,parents=None)
    last_data_node = first_data_node
    for data in datas[1:-1]:
        last_data_node = node.DataNode(data=data).add_parent(last_data_node)
    last_data_node = node.DataNode(data=datas[-1], children=None).add_parent(last_data_node)
    return first_data_node, last_data_node

In [33]:
def scores_from_path_data_graph(
    last_data_node,
    input_shape,
    first_data_node,
    train_inputs_as_array,
    train_labels_as_one_hot_encoded,
    validate_inputs_as_array,
    validate_labels_as_one_hot_encoded,
    epochs=10,
    callbacks=[
        keras.callbacks.EarlyStopping(patience=3, restore_best_weights=False, monitor='accuracy', min_delta=0.005, verbose=1),
        keras.callbacks.EarlyStopping(patience=4, restore_best_weights=False, monitor='val_accuracy', min_delta=0.005, verbose=1),
    ],
    verbose=1,
    batch_size=16
):
    
    assert input_shape == train_inputs_as_array.shape[1:] == validate_inputs_as_array.shape[1:], f"{input_shape = } {train_inputs_as_array.shape[1:] = } {validate_inputs_as_array.shape[1:] = }"
    keras_model = node.path_keras_model_from_last_data_node(
        last_data_node=last_data_node, 
        input_shape=input_shape,
        first_data_node=first_data_node,
    )
    keras_model.compile(
        optimizer='adam', 
        loss='categorical_crossentropy', 
        metrics=['accuracy'],
    )
    keras_model.summary()
    # print(f"{[train_inputs_as_array[0]] = }")
    # print(f"{train_inputs_as_array[0].shape = }")
    # print(f"{keras_model.predict( [ numpy.expand_dims(train_inputs_as_array[0], axis=0) ] ) = }")
    assert (batch_size & (batch_size-1) == 0) and batch_size != 0 ,f"internet says that batch size: {batch_size} should be a power of two"
    assert train_inputs_as_array.shape == ( *train_labels_as_one_hot_encoded.shape[:1] , *input_shape ), f"shape missmatch"
    history = keras_model.fit(
        x=train_inputs_as_array,
        y=train_labels_as_one_hot_encoded,
        validation_data=(validate_inputs_as_array,validate_labels_as_one_hot_encoded),
        epochs=epochs,
        batch_size=batch_size,
        validation_batch_size=batch_size,
        callbacks=callbacks,
        verbose=verbose,
    )
    score_by_metric = { metric: values[-1] for metric, values in history.history.items() }
    return score_by_metric

starting from the reference/control DCNN, 
keeping in mind we need to achieve 60% accuracy (test),
- DCNN reference
    - ~ 0.57 validation accuracy before 10 epochs
from little experiment we can assume that:
- we can easily:
    - add augmentation layer (rotation, shift, flip)
        - ~ 0.55 validation accurary before 10 epochs expect horizontal flip which reduce a little bit accuracy
    - reduced convolution kernel size
        - ~ 0.55 validation accurary before 10 epochs
    - reduced convolution features number
        - ~ 0.55 | 0.57 validation accurary before 10 epochs
- at the cost of some overfitting:
    - remove dropout layers
        - ~ 0.55 validation accurary before 10 epochs
- reduce validation accuracy and training time
    - half the number of training inputs
        - ~ 0.50 before 10 epochs
- did not work at all:
    - adding layer normalization
    - removing just batch normalization
    - batch size below 64 perform same

so we will genreate random convolution starting from reference.
    mutations will be about:
        - mutation convolution kernel size
        - mutating comvolution features number
        - mutating dropout ratio
    to speed up thing we will cut the number of input by half but add augmentation layer

In [34]:
import random
def mutated_dcnn_datas(dcnn_datas, number_of_mutations=1):
    def is_mutable_data(a_data):
        return hasattr(a_data, 'mutated')

    def get_indices_of_mutables_datas(datas):
        return list(
            index
            for index, data in enumerate(datas)
            if is_mutable_data(data)
        )
    assert get_indices_of_mutables_datas(dcnn_datas), f"no mutable data ? {dcnn_datas}"
    indices_of_data_to_mutate = random.sample(get_indices_of_mutables_datas(dcnn_datas), number_of_mutations)
    return list(
        data.mutated() if index in indices_of_data_to_mutate else data
        for index, data in enumerate(dcnn_datas)
    )

def expanded_dcnn_datas(
    dcnn_datas, 
    pool_of_layers=[
        data.MaxPoolingData, data.DropoutData, 
        data.BatchNormalizationData, data.LayerNormalizationData,
        lambda : data.ConvolutionData(features=512,shape=(3,3)), # we do not want to bottleneck dimensionality
    ],
):
    flatten_layer_index = None
    for index, a_data in enumerate(dcnn_datas):
        if isinstance(a_data, data.FlattenData):
            flatten_layer_index = index
            break
    flatten_layer_index = len(dcnn_datas)-1 if not flatten_layer_index else flatten_layer_index
    
    intercalation_index = random.randrange(flatten_layer_index)
    return dcnn_datas[:intercalation_index] + [random.choice(pool_of_layers)()] + dcnn_datas[intercalation_index:]

def shrank_dcnn_datas(
    dcnn_datas, 
):
    flatten_layer_index = None
    for index, a_data in enumerate(dcnn_datas):
        if isinstance(a_data, data.FlattenData):
            flatten_layer_index = index
            break
    flatten_layer_index = len(dcnn_datas) if not flatten_layer_index else flatten_layer_index
    removing_index = random.randrange(flatten_layer_index)
    return dcnn_datas[:removing_index] + dcnn_datas[removing_index+1:]

In [35]:
import time 
class TimeOut(keras.callbacks.Callback):
    def __init__(self, t0, timeout):
        super().__init__()
        self.t0 = t0
        self.timeout = timeout  # time in minutes

    def on_train_batch_end(self, batch, logs=None):
        if time.time() - self.t0 > self.timeout * 60:  # 58 minutes
            print(f"\nReached {(time.time() - self.t0) / 60:.3f} minutes of training, stopping")
            self.model.stop_training = True

callbacks = [TimeOut(t0=time.time(), timeout=58)]

In [36]:
import pickle
import os 
import constant
def save_datas_as_pickle(the_datas, destination_folder_path=constant.architecture_folder_path, file_name=None):
    file_name = f"architecture_data_{random.randint(1,999)}"
    if not os.path.exists(destination_folder_path):
        os.makedirs(destination_folder_path)
    destination_file_path = f"{destination_folder_path}{file_name}"
    with open(destination_file_path, 'wb') as handle:
        pickle.dump(the_datas, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(destination_file_path, 'rb') as handle:
        loaded_datas = pickle.load(handle)

In [37]:
def evolve_datas(
    train_inputs_as_array,
    train_labels_as_one_hot_encoded,
    validate_inputs_as_array,
    validate_labels_as_one_hot_encoded,
    initial_dcnn_datas,
    augmentation_datas,
    preprocessing_datas,
    validation_accuracy=None,
):
    for _ in range(20):
        better_datas, better_validation_accuracy = one_datas_evolution_cycle(
            train_inputs_as_array=train_inputs_as_array,
            train_labels_as_one_hot_encoded=train_labels_as_one_hot_encoded,
            validate_inputs_as_array=validate_inputs_as_array,
            validate_labels_as_one_hot_encoded=validate_labels_as_one_hot_encoded,
            initial_dcnn_datas=initial_dcnn_datas,
            augmentation_datas=augmentation_datas,
            preprocessing_datas=preprocessing_datas,
            validation_accuracy=validation_accuracy,
        )
        if better_validation_accuracy:
            saved_datas_file_path = f"{constant.architecture_folder_path}dcnn_validatation_accuracy_{int(better_validation_accuracy*100)}"
            save_datas_as_pickle(
                better_datas, 
                constant.architecture_folder_path, 
                saved_datas_file_path,
            )
    return saved_datas_file_path

def one_datas_evolution_cycle(
    train_inputs_as_array,
    train_labels_as_one_hot_encoded,
    validate_inputs_as_array,
    validate_labels_as_one_hot_encoded,
    initial_dcnn_datas,
    augmentation_datas,
    preprocessing_datas,
    validation_accuracy=None,
):
    retained_dcnn_datas = initial_dcnn_datas

    def validation_accuracy_from_dcnn_datas(dcnn_datas):
        pipeline_path_data_graph = path_data_graph_from_data_nodes( augmentation_datas + preprocessing_datas + dcnn_datas )
        return scores_from_path_data_graph(
            last_data_node=pipeline_path_data_graph[-1],
            input_shape=(48,48),
            first_data_node=pipeline_path_data_graph[0],
            train_inputs_as_array=train_inputs_as_array,
            train_labels_as_one_hot_encoded=train_labels_as_one_hot_encoded,
            validate_inputs_as_array=validate_inputs_as_array,
            validate_labels_as_one_hot_encoded=validate_labels_as_one_hot_encoded,
            epochs=40,
            callbacks=[
                keras.callbacks.EarlyStopping(patience=3, restore_best_weights=False, monitor='accuracy', min_delta=0.005, verbose=1),
                keras.callbacks.EarlyStopping(patience=4, restore_best_weights=False, monitor='val_accuracy', min_delta=0.005, verbose=1),
                TimeOut(t0=time.time(), timeout=2),
            ],
            verbose=1,
            batch_size=32,
        )['val_accuracy']

    if not validation_accuracy:
        validation_accuracy = validation_accuracy_from_dcnn_datas(dcnn_datas=initial_dcnn_datas)    
    # validation_accuracy = 0.1    
    # while True:
    saved_datas_file_path = None
    new_created_dcnn_datas = random.choice([mutated_dcnn_datas, expanded_dcnn_datas, shrank_dcnn_datas])
    print(f"try one evolution step {new_created_dcnn_datas = }")
    try:
        new_dcnn_datas = new_created_dcnn_datas(retained_dcnn_datas)
        new_validation_accuracy = validation_accuracy_from_dcnn_datas(new_dcnn_datas)
    except Exception as e:
        print(f"failed during mutation: {e}")
        print(f"{new_dcnn_datas = }")
    if new_validation_accuracy > validation_accuracy:
        print(f"\n new best val accuracy:{new_validation_accuracy = }")
        print(f"new best model:{new_dcnn_datas}")
        return new_dcnn_datas, new_validation_accuracy
    else:
        print("new model is not better")
    return None, None

In [38]:
def test():

    sampled_train_data_frame = fer.sampled_data_frame(
        train_data_frame,
        to_be_preserved_column_names=['emotion'],
        sampling_quantity=3000,
    )
    # print(f"{sampled_train_data_frame.info() = }")

    augmentation_datas = [
        data.data_from_keras_layer( keras.layers.RandomRotation, factor=0.005 ),
        data.data_from_keras_layer( keras.layers.RandomTranslation, height_factor=0.01, width_factor=0.01 ),
        data.data_from_keras_layer( keras.layers.RandomFlip, mode='horizontal' ),
    ]

    preprocessing_datas = [
        data.data_from_keras_layer( keras.layers.Reshape, target_shape=(48, 48, 1) ),
    ]

    dcnn_datas = [ # reduced features
        data.ConvolutionData(shape=(5,5),features=32,activation='elu'),data.BatchNormalizationData(),
        data.ConvolutionData(shape=(5,5),features=32,activation='elu'),data.BatchNormalizationData(),
        data.MaxPoolingData(shape=(2,2)), data.DropoutData(rate=0.4),
        data.ConvolutionData(shape=(3,3),features=64,activation='elu'),data.BatchNormalizationData(),
        data.ConvolutionData(shape=(3,3),features=64,activation='elu'),data.BatchNormalizationData(),
        data.MaxPoolingData(shape=(2,2)), data.DropoutData(rate=0.4),
        data.ConvolutionData(shape=(3,3),features=128,activation='elu'),data.BatchNormalizationData(),
        data.ConvolutionData(shape=(3,3),features=128,activation='elu'),data.BatchNormalizationData(),
        data.MaxPoolingData(shape=(2,2)), data.DropoutData(rate=0.5),
        data.FlattenData(),
        data.DenseData(units=128, activation='elu'),
        data.BatchNormalizationData(),
        data.DropoutData(rate=0.6),
        data.DenseData(units=7, activation='softmax'),
    ]

    # dcnn_datas = [ #reference
    #     data.ConvolutionData(shape=(5,5),features=64,activation='elu'),data.BatchNormalizationData(),
    #     data.ConvolutionData(shape=(5,5),features=64,activation='elu'),data.BatchNormalizationData(),
    #     data.MaxPoolingData(shape=(2,2)), data.DropoutData(rate=0.4),
    #     data.ConvolutionData(shape=(3,3),features=128,activation='elu'),data.BatchNormalizationData(),
    #     data.ConvolutionData(shape=(3,3),features=128,activation='elu'),data.BatchNormalizationData(),
    #     data.MaxPoolingData(shape=(2,2)), data.DropoutData(rate=0.4),
    #     data.ConvolutionData(shape=(3,3),features=256,activation='elu'),data.BatchNormalizationData(),
    #     data.ConvolutionData(shape=(3,3),features=256,activation='elu'),data.BatchNormalizationData(),
    #     data.MaxPoolingData(shape=(2,2)), data.DropoutData(rate=0.5),
    #     data.FlattenData(),
    #     data.DenseData(units=128, activation='elu'),
    #     data.BatchNormalizationData(),
    #     data.DropoutData(rate=0.6),
    #     data.DenseData(units=7, activation='softmax'),
    # ]
    train_inputs_as_array, train_labels_as_one_hot_encoded = keras_inputs_and_labels_from_data_frame(sampled_train_data_frame)
    validate_inputs_as_array, validate_labels_as_one_hot_encoded = keras_inputs_and_labels_from_data_frame(validate_data_frame)
    return evolve_datas(
        train_inputs_as_array=train_inputs_as_array,
        train_labels_as_one_hot_encoded=train_labels_as_one_hot_encoded,
        validate_inputs_as_array=validate_inputs_as_array,
        validate_labels_as_one_hot_encoded=validate_labels_as_one_hot_encoded,
        initial_dcnn_datas=dcnn_datas,
        augmentation_datas=augmentation_datas,
        preprocessing_datas=preprocessing_datas,
        # retained_validation_accuracies=retained_validation_accuracies,
    )
saved_datas_path = test()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 48, 48)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_1               │ (None, 48, 48)         │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_translation_1            │ (None, 48, 48)         │             0 │
│ (RandomTranslation)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip_1 (RandomFlip)      │ (None, 48, 48)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 48, 48, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 44, 44, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 44, 44, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 40, 40, 32)     │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 40, 40, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 20, 20, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 20, 20, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 18, 18, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 18, 18, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 6, 6, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 6, 6, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 4, 4, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 4, 4, 128)      │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 372,199 (1.42 MB)

 Trainable params: 371,047 (1.42 MB)

 Non-trainable params: 1,152 (4.50 KB)

Epoch 1/40
 50/898 ━━━━━━━━━━━━━━━━━━━━ 1:09 82ms/step - accuracy: 0.1226 - loss: 3.3905

KeyboardInterrupt: 

In [ ]:
import pickle
def load_datas_as_pickle(the_datas, destination_file_path):
    with open(destination_file_path, 'rb') as handle:
        loaded_datas = pickle.load(handle)
        try_to_make_data_graph_from_loaded_data = path_data_graph_from_data_nodes( loaded_datas )
        return loaded_data

In [ ]:
# cehck we saved anything
load_datas_as_pickle(saved_datas_path)

In [ ]:
# def evolve_dcnn(
#     train_image_series,
#     train_labels_as_one_hot_encoded,
#     validate_image_series,
#     validate_labels_as_one_hot_encoded,
#     initial_dcnn_datas,
#     augmentation_datas,
#     preprocessing_datas,
#     # retained_dcnn_datas=[],
#     # retained_validation_accuracies=[],
# ):
#     retained_dcnn_datas = initial_dcnn_datas

#     def validation_accuracy_from_dcnn_datas(dcnn_datas):
#         pipeline_path_data_graph = path_data_graph_from_data_nodes( augmentation_datas + preprocessing_datas + dcnn_datas )
#         return scores_from_path_data_graph(
#             last_data_node=pipeline_path_data_graph[-1],
#             input_shape=(48,48),
#             first_data_node=pipeline_path_data_graph[0],
#             train_inputs=get_all_inputs_as_array_from_matrix_series(train_image_series),
#             train_labels=keras.utils.to_categorical(train_labels_as_one_hot_encoded),
#             validate_inputs=get_all_inputs_as_array_from_matrix_series(validate_image_series),
#             validate_labels=keras.utils.to_categorical(validate_labels_as_one_hot_encoded),
#             epochs=40,
#             callbacks=[
#                 keras.callbacks.EarlyStopping(patience=3, restore_best_weights=False, monitor='accuracy', min_delta=0.005, verbose=1),
#                 keras.callbacks.EarlyStopping(patience=4, restore_best_weights=False, monitor='val_accuracy', min_delta=0.005, verbose=1),
#                 TimeOut(t0=time.time(), timeout=2),
#             ],
#             verbose=1,
#             batch_size=32,
#         )['val_accuracy']
        
#     validation_accuracy = validation_accuracy_from_dcnn_datas(dcnn_datas=initial_dcnn_datas)    
#     # validation_accuracy = 0.1    
#     # while True:
#     saved_datas_file_path = None
#     for _ in range(10):
#         print("try one mutation step")
#         try:
#             new_dcnn_datas = mutated_dcnn_datas(retained_dcnn_datas)
#             # print(f"\t{new_dcnn_datas = }")
#             new_validation_accuracy = validation_accuracy_from_dcnn_datas(new_dcnn_datas)
#             if new_validation_accuracy > validation_accuracy:
#                 print(f"\n new best val accuracy:{new_validation_accuracy = }")
#                 print(f"new best model:{new_dcnn_datas}")
#                 yield 
#                 # retained_dcnn_datas.append(new_dcnn_datas)
#                 # retained_validation_accuracies.append(new_validation_accuracy)
#                 saved_datas_file_path = f"{constant.architecture_folder_path}dcnn_validatation_accuracy_{int(validation_accuracy*100)}"
#                 save_datas_as_pickle(
#                     new_dcnn_datas, 
#                     constant.data_folder_path, 
#                     saved_datas_file_path,
#                 )
#                 retained_dcnn_datas = new_dcnn_datas
#                 validation_accuracy = new_validation_accuracy
#             else:
#                 print("no better model")
#         except Exception as e:
#             print(f"failed during mutation: {e}")
#             print(f"{new_dcnn_datas = }")
#             raise(e)
#         print("try one augmenting step")
#         try:
#             new_dcnn_datas = expanded_dcnn_datas(retained_dcnn_datas)
#             # print(f"\t{new_dcnn_datas = }")
#             new_validation_accuracy = validation_accuracy_from_dcnn_datas(new_dcnn_datas)
#             if new_validation_accuracy > validation_accuracy:
#                 print(f"\n new best val accuracy:{new_validation_accuracy = }")
#                 print(f"new best model:{new_dcnn_datas}")
#                 # retained_dcnn_datas.append(new_dcnn_datas)
#                 # retained_validation_accuracies.append(new_validation_accuracy)
#                 saved_datas_file_path = f"{constant.architecture_folder_path}dcnn_validatation_accuracy_{int(validation_accuracy*100)}"
#                 save_datas_as_pickle(
#                     new_dcnn_datas, 
#                     constant.data_folder_path, 
#                     saved_datas_file_path,
#                 )
#                 retained_dcnn_datas = new_dcnn_datas
#                 validation_accuracy = new_validation_accuracy
#             else:
#                 print("no better model")
#         except Exception as e:
#             print(f"failed during mutation: {e}")
#             print(f"{new_dcnn_datas = }")
#             raise(e)
#         print("try one reducing step")
#         try:
#             new_dcnn_datas = shrank_dcnn_datas(retained_dcnn_datas)
#             # print(f"\t{new_dcnn_datas = }")
#             new_validation_accuracy = validation_accuracy_from_dcnn_datas(new_dcnn_datas)
#             if new_validation_accuracy > validation_accuracy:
#                 print(f"\n new beretained_dcnn_datasst val accuracy:{new_validation_accuracy = }")
#                 print(f"new best model:{new_dcnn_datas}")
#                 # retained_dcnn_datas.append(new_dcnn_datas)
#                 # retained_validation_accuracies.append(new_validation_accuracy)
#                 saved_datas_file_path = f"{constant.architecture_folder_path}dcnn_validatation_accuracy_{int(validation_accuracy*100)}"
#                 save_datas_as_pickle(
#                     new_dcnn_datas, 
#                     constant.data_folder_path, 
#                     saved_datas_file_path,
#                 )
#                 retained_dcnn_datas = new_dcnn_datas
#                 validation_accuracy = new_validation_accuracy
#             else:
#                 print("no better model")
#         except Exception as e:
#             print(f"failed during mutation: {e}")
#             print(f"{new_dcnn_datas = }")
#             # raise(e)

#     return saved_datas_file_path